Installing python modules

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

# 1. Data cleaning

Loading real estate data into pandas dataframe

In [3]:
df=pd.read_csv("C:\\Users\\vande\\challenge-regression\\data\\real_estate_belgium.csv")

Loading and exploring the data

In [4]:
df.describe()
df.head() # No duplicates and no NAN's 

,Unnamed: 0,longitude,latitude,Postal_Code,Region,Province,Municipality,Price,Price_m2,Type_of_Property,Subtype_of_Property,State_of_the_Building,Number_of_Rooms,Living_Area,Fully_Equipped_Kitchen,Furnished,Open_fire,Terrace,Terrace_Area,Garden,Garden_Area,Number_of_Facades,Swimming_Pool,Disabled_Access,Lift
0,0,4.376134,51.248448,2000,2,8,Antwerpen,895000,3129,1,DUPLEX,0,3,286,1,0,0,1,30,0,0,2,0,0,0
1,1,4.361370,50.789871,1180,0,0,Uccle,685000,5393,1,PENTHOUSE,2,2,127,0,0,0,1,55,0,0,4,0,0,1
2,3,3.708325,51.003447,9052,2,9,Gent,429210,4292,1,APARTMENT,0,2,100,1,0,0,1,12,0,0,3,0,0,1
3,5,4.047008,50.941781,9300,2,9,Aalst,359000,197,0,HOUSE,5,3,122,0,0,0,1,30,1,1700,2,0,0,0
4,7,3.720262,51.053261,9000,2,9,Gent,560000,3684,0,HOUSE,6,4,152,0,0,0,1,23,0,0,2,0,0,1


# 2: Features engineering

Keep column features which have high correlation coefficients

In [5]:
df = df[["Municipality", "Price", "Living_Area", "Number_of_Rooms", "Fully_Equipped_Kitchen", "Terrace_Area"]]

In [6]:
df.describe()

,Price,Living_Area,Number_of_Rooms,Fully_Equipped_Kitchen,Terrace_Area
count,5.989000e+03,5989.000000,5989.000000,5989.000000,5989.000000
mean,4.023257e+05,131.432960,2.535148,0.278845,19.201870
std,2.224666e+05,70.267915,1.143061,0.448468,21.059055
min,3.000000e+04,17.000000,0.000000,0.000000,1.000000
25%,2.490000e+05,86.000000,2.000000,0.000000,7.000000
50%,3.450000e+05,110.000000,2.000000,0.000000,13.000000
75%,4.800000e+05,159.000000,3.000000,1.000000,24.000000
max,1.335000e+06,547.000000,7.000000,1.000000,215.000000


# 3: Data formatting

## Create XGBoost regression matrices

In [7]:
# Extract feature and target arrays

X, y = df.drop('Price', axis=1), df[['Price']]

In [8]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [9]:
X.dtypes

Municipality              category
Living_Area                  int64
Number_of_Rooms              int64
Fully_Equipped_Kitchen       int64
Terrace_Area                 int64
dtype: object

In [10]:
# split dataset into training and test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

In [11]:
# Create regression matrices

dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

## XGBoost regression

### Training data

In [12]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist", "device":"cuda"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

### Model evaluation with r², rmse (root mean squared error), and mae (mean absolute error)

In [ ]:
# model prediction

preds = model.predict(dtest_reg)

rmse = root_mean_squared_error(y_test, preds)

r2 = r2_score(y_test, preds)

mae = mean_absolute_error(y_test, preds)

print(f"R-squared: {r2:.2f}, RMSE: {rmse:.0f}, MAE: {mae:.0f} for test data in base model.")

R-squared: 0.69, RMSE: 121441, MAE: 79871 for test data in base model


In [37]:
print(preds, y_test)

[ 435225.84  168039.97  818524.9  ...  339885.47  375451.2  1101589.5 ]         Price
5787   499000
5406   229000
3296   965000
3731   545000
496    299000
...       ...
4220   395000
4893   380000
5694   399000
5222   320000
2379  1325000

[1198 rows x 1 columns]


### Using Validation Sets During Training

In [15]:
# set up the parameters

params = {"objective": "reg:squarederror", "tree_method": "hist", "device":"cuda"}
n = 100

In [16]:
# setup list of two tuples that each contain two elements. 
# The first element is the array for the model to evaluate, and the second is the array’s name.

evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

In [17]:
# pass this array to the evals parameter of xgb.train, we will see the model performance after each boosting round:

evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10 # Every ten rounds
)

[0]	train-rmse:178015.25831	validation-rmse:177975.54085
[10]	train-rmse:93136.04399	validation-rmse:120108.19564
[20]	train-rmse:82401.22590	validation-rmse:119377.05310
[30]	train-rmse:75292.63055	validation-rmse:119298.63039
[40]	train-rmse:69632.97701	validation-rmse:119936.64194
[50]	train-rmse:64085.33069	validation-rmse:120574.88242
[60]	train-rmse:59694.25142	validation-rmse:121141.51884
[70]	train-rmse:55618.63816	validation-rmse:121816.02915
[80]	train-rmse:51436.35349	validation-rmse:122265.98600
[90]	train-rmse:47755.32680	validation-rmse:122438.89363
[99]	train-rmse:46293.61676	validation-rmse:122406.58757


## XGBoost early stopping

Generally, the more rounds there are, the more XGBoost tries to minimize the loss. 

But this doesn’t mean the loss will always go down. Let’s try with 5000 boosting rounds with the verbosity of 500:

In [18]:
params = {"objective": "reg:squarederror", "tree_method": "hist", "device":"cuda"}
n = 5000

evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=250 # Every ten rounds
)


[0]	train-rmse:178015.25831	validation-rmse:177975.54085


[250]	train-rmse:24243.73656	validation-rmse:124788.42676
[500]	train-rmse:13150.22927	validation-rmse:126379.28873
[750]	train-rmse:10249.00327	validation-rmse:127134.19673
[1000]	train-rmse:9532.54570	validation-rmse:127294.27331
[1250]	train-rmse:9347.26903	validation-rmse:127416.80131
[1500]	train-rmse:9296.95162	validation-rmse:127467.77404
[1750]	train-rmse:9282.82692	validation-rmse:127495.95251
[2000]	train-rmse:9278.87116	validation-rmse:127512.27162
[2250]	train-rmse:9277.77589	validation-rmse:127519.09427
[2500]	train-rmse:9277.40737	validation-rmse:127522.64952
[2750]	train-rmse:9277.31155	validation-rmse:127524.10784
[3000]	train-rmse:9277.28076	validation-rmse:127524.95102
[3250]	train-rmse:9277.27009	validation-rmse:127525.41610
[3500]	train-rmse:9277.26735	validation-rmse:127525.70015
[3750]	train-rmse:9277.26664	validation-rmse:127525.84853
[4000]	train-rmse:9277.26641	validation-rmse:127525.88860
[4250]	train-rmse:9277.26630	validation-rmse:127525.94380
[4500]	train-r

We want the golden middle: a model that learned just enough patterns in training that it gives the highest performance on the validation set. So, how do we find the perfect number of boosting rounds, then?

We will use a technique called early stopping. Early stopping forces XGBoost to watch the validation loss, and if it stops improving for a specified number of rounds, it automatically stops training.

This means we can set as high a number of boosting rounds as long as we set a sensible number of early stopping rounds.

In [19]:
params = {"objective": "reg:squarederror", "tree_method": "hist", "device":"cuda"}
n = 10000

evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
   # Activate early stopping
   early_stopping_rounds = 50

)


[0]	train-rmse:178015.25831	validation-rmse:177975.54085
[10]	train-rmse:93136.04399	validation-rmse:120108.19564
[20]	train-rmse:82401.22590	validation-rmse:119377.05310
[30]	train-rmse:75292.63055	validation-rmse:119298.63039
[40]	train-rmse:69632.97701	validation-rmse:119936.64194
[50]	train-rmse:64085.33069	validation-rmse:120574.88242
[60]	train-rmse:59694.25142	validation-rmse:121141.51884
[66]	train-rmse:56942.23890	validation-rmse:121441.39700


## XGBoost Cross-Validation

Since we try to find the best value of a hyperparameter by comparing the validation performance of the model on the test set, 
we will end up with a model that is configured to perform well only on that particular test set. 

Instead, we want a model that performs well across the board — on any test set we throw at it.

A possible workaround is splitting the data into three sets. The model trains on the first set, the second set is used for evaluation and hyperparameter tuning, and the third is the final one we test the model before production.

But when data is limited, splitting data into three sets will make the training set sparse, which hurts model performance.

The solution to all these problems is cross-validation. In cross-validation, we still have two sets: training and testing.

In [ ]:
# While the test set waits in the corner, we split the training into 3, 5, 7, or k splits or folds. 
# Then, we train the model k times. Each time, we use k-1 parts for training and the final kth part for validation. 
# This process is called k-fold cross-validation:

params = {"objective": "reg:squarederror", "tree_method": "hist", "device":"cuda"}
n = 1000

results = xgb.cv(
   params, dtrain_reg,
   num_boost_round=n, 
   nfold=5, # specify the number of splits
   early_stopping_rounds=20
)

In [21]:
# It has the same number of rows as the number of boosting rounds. 

# Each row is the average of all splits for that round. So, to find the best score, we take the minimum of the test-rmse-mean column:

best_rmse = results['test-rmse-mean'].min()

print(best_rmse)

123513.51370391701


In [34]:
# model prediction

preds = model.predict(dtest_reg)

rmse = root_mean_squared_error(y_test, preds)

r2 = r2_score(y_test, preds)

mae = mean_absolute_error(y_test, preds)

print(f"R-squared: {r2:.2f}, RMSE: {rmse:.0f}, MAE: {mae:.0f} for test data in base model.")

R-squared: 0.69, RMSE: 121441, MAE: 79871 for test data in base model.
